In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
df_movies = pd.read_csv(r'Movies\movies_dataset.csv')

C:\Users\debor\AppData\Local\Temp\ipykernel_5096\4200754764.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv(r'Movies\movies_dataset.csv')


In [3]:
# Se convierte la comuna 'popularity' a tipo float, para poder pasar de csv a parquet
df_movies['popularity'] = pd.to_numeric(df_movies['popularity'], errors='coerce')

In [4]:
df_movies.to_parquet(r'Movies\df_movies.parquet', index=False)

Desanidado 'belongs_to_collection'

In [5]:
# Se verifican tipos de datos: 'str' indica que es una cadena JSON
print(df_movies['belongs_to_collection'].apply(type).unique())

[<class 'str'> <class 'float'>]


In [6]:
"""Se convertirán las cadenas JSON a 'dict', para manejarlas en Python
    Luego se extraen los valores de las claves de la columna 'belongs_to_collection'
    y con esos valores se crean nuevas columnas para agregarlas al DataSet
    """
def convert_to_dict(x): 
    if isinstance(x, str):
            return ast.literal_eval(x)
    return x

# Se aplica la funcion a la columna 'belongs_to_collection'
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].apply(convert_to_dict)

# Se crean nuevas columnas con los valores extríados de los diccionarios de 'belongs_to_collection'
df_movies['collection_id'] = df_movies['belongs_to_collection'].apply(lambda x: x['id'] if isinstance(x, dict) and 'id' in x else None)
df_movies['collection_name'] = df_movies['belongs_to_collection'].apply(lambda x: x['name'] if isinstance(x, dict) and 'name' in x else None)
df_movies['poster_path_collection'] = df_movies['belongs_to_collection'].apply(lambda x: x['poster_path'] if isinstance(x, dict) and 'poster_path' in x else None)
df_movies['backdrop_path'] = df_movies['belongs_to_collection'].apply(lambda x: x['backdrop_path'] if isinstance(x, dict) and 'backdrop_path' in x else None)

print(df_movies[['collection_id','collection_name','poster_path','backdrop_path']].head(3))


   collection_id            collection_name                       poster_path  \
0        10194.0       Toy Story Collection  /rhIRbceoE9lR4veEXuwCC2wARtG.jpg   
1            NaN                       None  /vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg   
2       119050.0  Grumpy Old Men Collection  /6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg   

                      backdrop_path  
0  /9FBwqcd9IRruEDUrTdcaafOMKUq.jpg  
1                              None  
2  /hypTnLot2z8wpFS7qwsQHW1uV8u.jpg  


In [7]:
df_movies.shape

(45466, 28)

In [8]:
# Se verifican los nulos existentes en cada nueva columna
btc_nulls = df_movies[['collection_id','collection_name','poster_path','backdrop_path']].isnull().sum()
print(btc_nulls)

collection_id      40975
collection_name    40975
poster_path          386
backdrop_path      42203
dtype: int64


In [9]:
# Se eliminan los nulos de las columnas desanidadas
df_movies.dropna(subset=['collection_id','collection_name','poster_path','backdrop_path'], inplace=True)

In [10]:
# Se elimina la columna inicial que se desanidó
df_movies = df_movies.drop(columns=['belongs_to_collection'])

In [11]:
# Se verifica el tipo de dato de la columna 'collection_id'
print(df_movies['collection_id'].apply(type).unique())

[<class 'float'>]


In [12]:
# Se cambia el tipo de dato de la columna 'collection_id' a tipo 'int'
df_movies['collection_id'] = df_movies['collection_id'].astype(int)
print(df_movies['collection_id'].apply(type).unique())


[<class 'int'>]


In [13]:
df_movies.shape

(3261, 27)

Desanidado 'genres'

In [14]:
# Se verifican tipos de datos: 'str' indica que es una cadena JSON
print(df_movies['genres'].apply(type).unique())

[<class 'str'>]


In [15]:
"""Convertimos las listas de str a dict.
  creamos las dos nuevas columnas 'genres_id' y 'genres_name' con sus valores
  las concatenamos para formar un df temporal
  y luego lo unimos a df_movies
    """
    
def convert_to_list_of_dicts(x):
    if isinstance(x, str):
            return ast.literal_eval(x)
    return x

# Se aplica la funcion a la columna 'genres'
df_movies['genres'] = df_movies['genres'].apply(convert_to_list_of_dicts)

# Se crean nuevas columnas con los valores extríados de los diccionarios de 'genres'
df_movies['genres_id'] = df_movies['genres'].apply(lambda x: [d['id'] for d in x if isinstance(d, dict) and 'id' in d])
df_movies['genres_name'] = df_movies['genres'].apply(lambda x: [d['name'] for d in x if isinstance(d, dict) and 'name' in d])

# Se crean df temporales a partir de las listas de 'genres_id' y 'genres_name'
# Luego 'stack' convierte esas listas en valores individuales
# Se concatenan los df temporales en uno solo que contedrá las columnas 'genres_id' y 'genres_name'
temp_df = pd.concat([pd.DataFrame(df_movies['genres_id'].tolist(), index=df_movies.index).stack().reset_index(level=1, drop=True),
                     pd.DataFrame(df_movies['genres_name'].tolist(), index=df_movies.index).stack().reset_index(level=1, drop=True)], axis=1)
temp_df.columns = ['genres_id', 'genres_name']

# Se une el df temporal con df_movies (seeliminaron las columnas originales)
df_movies = df_movies.drop(columns=['genres_id', 'genres_name']).join(temp_df).reset_index(drop=True)

print(df_movies[['genres_id','genres_name']].head(3))


   genres_id genres_name
0       16.0   Animation
1       35.0      Comedy
2    10751.0      Family


In [16]:
df_movies.shape

(8331, 29)

In [17]:
# Se verifican los nulos existentes en cada nueva columna
genres_nulls = df_movies[['genres_id','genres_name']].isnull().sum()
print(genres_nulls)

genres_id      26
genres_name    26
dtype: int64


In [18]:
# Se eliminan los nulos de las columnas desanidadas
df_movies.dropna(subset=['genres_id','genres_name'], inplace=True)

In [19]:
# Se elimina la columna inicial que se desanidó
df_movies = df_movies.drop(columns=['genres'])

In [20]:
# Se verifica el tipo de dato de la columna 'genres_id'
print(df_movies['genres_id'].apply(type).unique())

[<class 'float'>]


In [21]:
# Se cambia el tipo de dato de la columna 'genres_id' al tipo 'int'
df_movies['genres_id'] = df_movies['genres_id'].astype(int)
print(df_movies['genres_id'].apply(type).unique())

[<class 'int'>]


In [22]:
df_movies.shape

(8305, 28)

Desanidado 'production_companies'

In [23]:
"""Convertimos las listas de str a dict.
  creamos las dos nuevas columnas 'prod_comp_name' y 'prod_comp_id' con sus valores
  las concatenamos para formar un df temporal
  y unimos el df temporal a df_movies
    """

def convert_to_list_of_dicts(x):
    if isinstance(x, str):
            return ast.literal_eval(x)
    elif isinstance(x, list):
        return x
    return []

# Se aplica la funcion a la columna 'production_companies'
df_movies['production_companies'] = df_movies['production_companies'].apply(convert_to_list_of_dicts)

# Se crean nuevas columnas con los valores extríados de los diccionarios de 'production_companies'
df_movies['prod_comp_name'] = df_movies['production_companies'].apply(lambda x: [d['name'] for d in x if isinstance(d, dict) and 'name' in d]if isinstance(x, list) else [])
df_movies['prod_comp_id'] = df_movies['production_companies'].apply(lambda x: [d['id'] for d in x if isinstance(d, dict) and 'id' in d]if isinstance(x, list) else [])

# Se crean df temporales a partir de las listas de 'prod_comp_name' y 'prod_comp_id'
# Luego 'stack' comvierte esas listas en valores individuales
# Se concatenan los df temporales en uno solo que contendrá las columnas 'prod_comp_name' y 'prod_comp_id'
temp_df = pd.concat([pd.DataFrame(df_movies['prod_comp_name'].tolist(), index=df_movies.index).stack().reset_index(level=1, drop=True),
                     pd.DataFrame(df_movies['prod_comp_id'].tolist(), index=df_movies.index).stack().reset_index(level=1, drop=True)], axis=1)
temp_df.columns = ['prod_comp_name', 'prod_comp_id']

# Se une el DataFrame temporal con el original
df_movies = df_movies.drop(columns=['prod_comp_id', 'prod_comp_name']).join(temp_df).reset_index(drop=True)

print(df_movies[['prod_comp_name', 'prod_comp_id']].head(10))

            prod_comp_name  prod_comp_id
0  Pixar Animation Studios           3.0
1  Pixar Animation Studios           3.0
2  Pixar Animation Studios           3.0
3             Warner Bros.        6194.0
4           Lancaster Gate       19464.0
5             Warner Bros.        6194.0
6           Lancaster Gate       19464.0
7    Sandollar Productions        5842.0
8      Touchstone Pictures        9195.0
9           United Artists          60.0


In [24]:
df_movies.shape

(18944, 30)

In [25]:
# Se verifican los nulos existentes en cada nueva columna
prod_nulls = df_movies[['prod_comp_name','prod_comp_id']].isnull().sum()
print(prod_nulls)

prod_comp_name    614
prod_comp_id      614
dtype: int64


In [26]:
# Se eliminan los nulos de las columnas desanidadas
df_movies.dropna(subset=['prod_comp_name','prod_comp_id'], inplace=True)

In [27]:
# Se elimina la columna inicial que se desanidó
df_movies = df_movies.drop(columns=['production_companies'])

In [28]:
# Se verifica el tipo de dato de la columna 'prod_comp_id'
print(df_movies['prod_comp_id'].apply(type).unique())

[<class 'float'>]


In [29]:
# Se cambia el tipo de dato de la columna 'prod_comp_id' al tipo 'int'
df_movies['prod_comp_id'] = df_movies['prod_comp_id'].astype(int)
print(df_movies['prod_comp_id'].apply(type).unique())

[<class 'int'>]


In [30]:
df_movies.shape

(18330, 29)

Desanidado 'production_countries'

In [31]:
# Se verifican tipos de datos: 'str' indica que es una cadena JSON
print(df_movies['production_countries'].apply(type).unique())

[<class 'str'>]


In [32]:
"""Convertimos las listas de str a dict.
  creamos las dos nuevas columnas 'genres_id' y 'genres_name' con sus valores
  las concatenamos para formar un df temporal
  y unimos el df temporal a df_movies
    """

def convert_to_list_of_dicts(x):
    if isinstance(x, str):
            return ast.literal_eval(x)
    elif isinstance(x, list):
        return x
    return []

# Se aplica la funcion a la columna 'production_countries'
df_movies['production_countries'] = df_movies['production_countries'].apply(convert_to_list_of_dicts)

# Se crean nuevas columnas con los valores extríados de los diccionarios de 'production_countries'
df_movies['iso'] = df_movies['production_countries'].apply(lambda x: [d['iso_3166_1'] for d in x if isinstance(d, dict) and 'iso_3166_1' in d]if isinstance(x, list) else [])
df_movies['prod_country_name'] = df_movies['production_countries'].apply(lambda x: [d['name'] for d in x if isinstance(d, dict) and 'name' in d]if isinstance(x, list) else [])

# Se crean df temporales a partir de las listas de 'iso' y 'prod_country_name'
# Luego 'stack' comvierte esas listas en valores individuales
# Se concatenan los df temporales en uno solo que contendrá las columnas 'iso' y 'prod_country_name'
temp_df = pd.concat([pd.DataFrame(df_movies['iso'].tolist(), index=df_movies.index).stack().reset_index(level=1, drop=True),
                     pd.DataFrame(df_movies['prod_country_name'].tolist(), index=df_movies.index).stack().reset_index(level=1, drop=True)], axis=1)
temp_df.columns = ['iso', 'prod_country_name']

# Se une el df temporal con df_movies
df_movies = df_movies.drop(columns=['iso', 'prod_country_name']).join(temp_df).reset_index(drop=True)

print(df_movies[['prod_country_name', 'iso']].head(10))

          prod_country_name iso
0  United States of America  US
1  United States of America  US
2  United States of America  US
3  United States of America  US
4  United States of America  US
5  United States of America  US
6  United States of America  US
7  United States of America  US
8  United States of America  US
9            United Kingdom  GB


In [33]:
df_movies.shape

(27319, 31)

In [34]:
# Se verifican los nulos existentes en cada nueva columna
country_nulls = df_movies[['prod_country_name','iso']].isnull().sum()
print(country_nulls)

prod_country_name    98
iso                  98
dtype: int64


In [35]:
# Se eliminan los nulos de las columnas desanidadas
df_movies.dropna(subset=['prod_country_name','iso'], inplace=True)

In [36]:
# Se elimina la columna inicial que se desanidó
df_movies = df_movies.drop(columns=['production_countries'])

In [37]:
df_movies.shape

(27221, 30)

Desanidado de 'spoken_languages'

In [38]:
# Se verifican tipos de datos: 'str' indica que es una cadena JSON
print(df_movies['spoken_languages'].apply(type).unique())

[<class 'str'>]


In [39]:
"""Convertimos las listas de str a dict.
  creamos las dos nuevas columnas 'genres_id' y 'genres_name' con sus valores
  las concatenamos para formar un df temporal
  y unimos el df temporal a df_movies
    """
    
def convert_to_list_of_dicts(x):
    if isinstance(x, str):
            return ast.literal_eval(x)
    return x

# Se aplica la funcion a la columna 'spoken_languages'
df_movies['spoken_languages'] = df_movies['spoken_languages'].apply(convert_to_list_of_dicts)

# Se crean nuevas columnas con los valores extríados de los diccionarios de 'spoken_languages'
df_movies['iso_language'] = df_movies['spoken_languages'].apply(lambda x: [d['iso_639_1'] for d in x if isinstance(d, dict) and 'iso_639_1' in d])
df_movies['language'] = df_movies['spoken_languages'].apply(lambda x: [d['name'] for d in x if isinstance(d, dict) and 'name' in d])

# Se crean df temporales a partir de las listas de 'iso_language' y 'language'
# Luego 'stack' comvierte esas listas en valores individuales
# Se concatenan los df temporales en uno solo que contendrá las columnas 'iso_language' y 'language'
temp_df = pd.concat([pd.DataFrame(df_movies['iso_language'].tolist(), index=df_movies.index).stack().reset_index(level=1, drop=True),
                     pd.DataFrame(df_movies['language'].tolist(), index=df_movies.index).stack().reset_index(level=1, drop=True)], axis=1)
temp_df.columns = ['iso_language', 'language']

# Se une el df temporal con df_movies
df_movies = df_movies.drop(columns=['iso_language', 'language']).join(temp_df).reset_index(drop=True)

print(df_movies[['language', 'iso_language']].head(10))

  language iso_language
0  English           en
1  English           en
2  English           en
3  English           en
4  English           en
5  English           en
6  English           en
7  English           en
8  English           en
9  English           en


In [40]:
df_movies.shape

(43746, 32)

In [41]:
# Se verifican los nulos existentes en cada nueva columna
laguage_nulls = df_movies[['language','iso_language']].isnull().sum()
print(laguage_nulls)

language        39
iso_language    39
dtype: int64


In [42]:
# Se eliminan los nulos de las columnas desanidadas
df_movies.dropna(subset=['language','iso_language'], inplace=True)

In [43]:
# Se elimina la columna inicial que se desanidó
df_movies = df_movies.drop(columns=['spoken_languages'])

In [44]:
df_movies.shape

(43707, 31)

In [45]:
# Se verifica el tipo de dato de las columnas 'revenue' y 'budget'
print(df_movies[['revenue', 'budget']].dtypes)

revenue    float64
budget      object
dtype: object


In [46]:
# Se cambia el tipo de dato de la columna 'revenue' al tipo 'int'
df_movies['revenue'] = df_movies['revenue'].astype(int)
print(df_movies['revenue'].apply(type).unique())

[<class 'int'>]


In [47]:
# Se cambia el tipo de dato de la columna 'budget' al tipo 'int'
df_movies['budget'] = df_movies['budget'].astype(int)
print(df_movies['budget'].apply(type).unique())

[<class 'int'>]


In [48]:
# Corroboro
print(df_movies[['revenue', 'budget']].dtypes)

revenue    int64
budget     int64
dtype: object


In [49]:
# Valores nulos de los campos revenue, budget rellenados por el número 0.
df_movies['revenue'].fillna(0)
df_movies['budget'].fillna(0)

print(df_movies[['revenue', 'budget']])

         revenue    budget
0      373554033  30000000
1      373554033  30000000
2      373554033  30000000
3              0         0
4              0         0
...          ...       ...
43741          0         0
43742          0         0
43743          0         0
43744          0         0
43745          0         0

[43707 rows x 2 columns]


In [50]:
# Se eliminan los valores nulos en 'release_date'
df_movies.dropna(subset=['release_date'], inplace=True)


In [51]:
# Corroboro
release_nulls = df_movies['release_date'].isnull().sum()
print(release_nulls)

0


In [52]:
df_movies.shape

(43701, 31)

In [53]:
# Se convierte la columna 'release_date' a fecha y hora
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], errors='coerce')

# Se crea la columna 'release_year' conteniendo el año de estreno
df_movies['release_year'] = df_movies['release_date'].dt.year.astype('Int64')

print(df_movies[['release_date', 'release_year']])

      release_date  release_year
0       1995-10-30          1995
1       1995-10-30          1995
2       1995-10-30          1995
3       1995-12-22          1995
4       1995-12-22          1995
...            ...           ...
43741   1967-03-15          1967
43742   1967-09-01          1967
43743   1967-09-01          1967
43744   1969-05-29          1969
43745   1976-10-01          1976

[43701 rows x 2 columns]


In [54]:
# Chequeo el tipo de datos de las columnas 'release_date' y 'release_year'
print(df_movies[['release_date', 'release_year']].dtypes)

release_date    datetime64[ns]
release_year             Int64
dtype: object


In [55]:
df_movies.shape

(43701, 32)

In [56]:
# Calculo del retorno y se crea la columna 'return'
df_movies['return'] = df_movies['revenue'] / df_movies['budget']
df_movies['return'].fillna(0, inplace=True)  # Se rellenan los valores nulos con 0

print(df_movies[['revenue', 'budget', 'return']])


         revenue    budget     return
0      373554033  30000000  12.451801
1      373554033  30000000  12.451801
2      373554033  30000000  12.451801
3              0         0   0.000000
4              0         0   0.000000
...          ...       ...        ...
43741          0         0   0.000000
43742          0         0   0.000000
43743          0         0   0.000000
43744          0         0   0.000000
43745          0         0   0.000000

[43701 rows x 3 columns]


C:\Users\debor\AppData\Local\Temp\ipykernel_5096\828371204.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_movies['return'].fillna(0, inplace=True)  # Se rellenan los valores nulos con 0


In [57]:
# Se eliminan las columnas que no serán utilizadas
columnas_a_descartar = ['video', 'imdb_id', 'adult', 'original_title','poster_path', 'homepage' ]
df_movies = df_movies.drop(columns=columnas_a_descartar)
df_movies.shape


(43701, 27)

In [58]:
# Se verifica si quedan nulos
nulls = df_movies[['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'collection_id', 'collection_name',
       'poster_path_collection', 'backdrop_path', 'genres_id', 'genres_name',
       'prod_comp_name', 'prod_comp_id', 'iso', 'prod_country_name',
       'iso_language', 'language', 'release_year', 'return']].isnull().sum()
print(nulls)

budget                        0
id                            0
original_language             0
overview                     40
popularity                    0
release_date                  0
revenue                       0
runtime                       6
status                        0
tagline                   11402
title                         0
vote_average                  0
vote_count                    0
collection_id                 0
collection_name               0
poster_path_collection      159
backdrop_path                 0
genres_id                     0
genres_name                   0
prod_comp_name                0
prod_comp_id                  0
iso                           0
prod_country_name             0
iso_language                  0
language                      0
release_year                  0
return                        0
dtype: int64


In [59]:
# Se eliminan los nulos
df_movies.dropna(subset=['overview','tagline','poster_path_collection'], inplace=True)

In [60]:
# Se le cambia el nombre a la columna 'id' a 'id_movie'
df_movies.rename(columns={'id': 'id_movie'}, inplace=True)

In [61]:
df_movies.shape

(32189, 27)

In [62]:
# Se le cambia el tipod e dato a la columna 'id_movie' a 'int'
df_movies['id_movie'] = df_movies['id_movie'].astype(int)

In [63]:
df_movies.to_parquet(r'Movies\df_movies_mod.parquet', engine='auto', compression='snappy')

Credits

In [64]:
df_credits = pd.read_csv(r'Movies\credits.csv')

Desanidado de 'Cast'

In [65]:
df_credits.shape

(45476, 3)

In [66]:
"""Se convierten las listas de str JSON a lista de dict.
  Se crean dos nuevas columnas: 'character' y 'genres_name' con sus valores,
  las concatenamos para formar un df temporal
  y unimos el df temporal a df_movies
    """
    
def convert_to_list_of_dicts(x):
    if isinstance(x, str):
            return ast.literal_eval(x)
    return x

# Se aplica la función a la columna 'cast'
df_credits['cast'] = df_credits['cast'].apply(convert_to_list_of_dicts)

# Se extraen los valores de 'character' y 'name' y se crean dos nuevas columnas de listas
df_credits['character'] = df_credits['cast'].apply(lambda x: [d['character'] for d in x if isinstance(d, dict) and 'character' in d])
df_credits['name'] = df_credits['cast'].apply(lambda x: [d['name'] for d in x if isinstance(d, dict) and 'name' in d])

# Las listas se pasan a df
character_df = pd.DataFrame(df_credits['character'].tolist(), index=df_credits.index).stack().reset_index(level=1, drop=True)
name_df = pd.DataFrame(df_credits['name'].tolist(), index=df_credits.index).stack().reset_index(level=1, drop=True)

# Se concatenan esos df y crean un solo df temporal y se renombran las columnas
temp_df = pd.concat([character_df, name_df], axis=1)
temp_df.columns = ['character_new', 'actor_name']

# Se une el df temporal a df_credits
df_credits = df_credits.join(temp_df).reset_index(drop=True)

print(df_credits[['character_new', 'actor_name']].head(10))

             character_new         actor_name
0            Woody (voice)          Tom Hanks
1   Buzz Lightyear (voice)          Tim Allen
2  Mr. Potato Head (voice)        Don Rickles
3       Slinky Dog (voice)         Jim Varney
4              Rex (voice)      Wallace Shawn
5             Hamm (voice)  John Ratzenberger
6          Bo Peep (voice)        Annie Potts
7             Andy (voice)        John Morris
8              Sid (voice)    Erik von Detten
9       Mrs. Davis (voice)     Laurie Metcalf


In [67]:
df_credits.shape

(564892, 7)

In [68]:
# Se verifican los nulos existentes en cada nueva columna
cast_nulls = df_credits[['character_new', 'actor_name']].isnull().sum()
print(cast_nulls)

character_new    2418
actor_name       2418
dtype: int64


In [69]:
# Se eliminan los nulos
df_credits.dropna(subset=['character_new','actor_name'], inplace=True)

In [70]:
df_credits = df_credits.drop(columns=['cast','character','name'])

In [71]:
df_credits.shape

(562474, 4)

Desanidado de 'crew'

In [72]:
def convert_to_list_of_dicts(x):
    if isinstance(x, str):
            return ast.literal_eval(x)
    return x

df_credits['crew'] = df_credits['crew'].apply(convert_to_list_of_dicts)

df_credits['crew_name'] = df_credits['crew'].apply(lambda x: [d['name'] for d in x if isinstance(d, dict) and 'name' in d])
df_credits['crew_job'] = df_credits['crew'].apply(lambda x: [d['job'] for d in x if isinstance(d, dict) and 'job' in d])

crew_name_df = pd.DataFrame(df_credits['crew_name'].tolist(), index=df_credits.index).stack().reset_index(level=1, drop=True)
crew_job_df = pd.DataFrame(df_credits['crew_job'].tolist(), index=df_credits.index).stack().reset_index(level=1, drop=True)

temp_df = pd.concat([crew_name_df, crew_job_df], axis=1)
temp_df.columns = ['director_name', 'crew_job_new']

df_credits = df_credits.join(temp_df).reset_index(drop=True)

print(df_credits[['director_name', 'crew_job_new']].head(10))

      director_name        crew_job_new
0     John Lasseter            Director
1       Joss Whedon          Screenplay
2    Andrew Stanton          Screenplay
3        Joel Cohen          Screenplay
4      Alec Sokolow          Screenplay
5     Bonnie Arnold            Producer
6        Ed Catmull  Executive Producer
7  Ralph Guggenheim            Producer
8        Steve Jobs  Executive Producer
9       Lee Unkrich              Editor


In [73]:
df_credits.shape

(9479469, 8)

In [75]:
credits_nulls = df_credits[['director_name','crew_job_new']].isnull().sum()
print(credits_nulls)

director_name    1806
crew_job_new     1806
dtype: int64


In [76]:
df_credits.dropna(subset=['director_name','crew_job_new'], inplace=True)

In [77]:
df_credits = df_credits.drop(columns=['crew','crew_name','crew_job'])

In [78]:
df_credits.shape

(9477663, 5)

In [79]:
# Se le cambia el nombre a la columna 'id' a 'id_movie_credits'
df_credits.rename(columns={'id': 'id_movie_credits'}, inplace=True)

In [80]:
df_credits.head(1)

,id_movie_credits,character_new,actor_name,director_name,crew_job_new
0,862,Woody (voice),Tom Hanks,John Lasseter,Director


In [81]:
df_credits.to_parquet(r'Movies\df_credits_mod.parquet', engine='auto', compression='snappy')

Creación de un nuevo DataFrame con la data necesaria de 'credits' para la función get_director()

In [87]:
credits_director = pd.read_parquet(r'Movies\df_credits_mod.parquet', engine='auto')

In [88]:
# Se filtra el df 'credits_director' para obtener solo los directores
credits_director = credits_director[credits_director['crew_job_new'] == 'Director']

In [89]:
# Se seleccionan las columnas 'id_movie_credits' y 'director_name'
credits_director = credits_director[['id_movie_credits', 'director_name']]

In [90]:
# Se renombra la columna 'id_movie_credits'
credits_director = credits_director.rename(columns={'id_movie_credits': 'id_movie'})

In [93]:
# Se borran duplicados
credits_director = credits_director.drop_duplicates(subset='id_movie')

In [94]:
print(credits_director)


         id_movie     director_name
0             862     John Lasseter
1381         8844      Joe Johnston
1794        15602     Howard Deutch
1822        31357   Forest Whitaker
1927        11862     Charles Shyer
...           ...               ...
9479246     30840        John Irvin
9479291    439050  Hamid Nematollah
9479318    111109          Lav Diaz
9479384     67758    Mark L. Lester
9479459    227506  Yakov Protazanov

[42568 rows x 2 columns]


In [95]:
# Se cambia el tipode dato de la columna 'id_movie' a 'int'
credits_director['id_movie'] = credits_director['id_movie'].astype(int)

In [96]:
credits_director.to_parquet(r'Movies\df_credits_director.parquet', engine='auto', compression='snappy')

Creación de un nuevo DataFrame con la data necesaria de 'credits' para la función get_actor()

In [97]:
credits_actor = pd.read_parquet(r'Movies\df_credits_mod.parquet', engine='auto')

In [99]:
# Se filtra el df 'credits_actor' para obtener solo las filas con información de actores
credits_actor = credits_actor[['id_movie_credits', 'actor_name']]

In [100]:
# Se renombra la columna 
credits_actor = credits_actor.rename(columns={'id_movie_credits': 'id_movie'})

In [104]:
# Eliminar filas donde 'actor' es NaN o vacío 
credits_actor = credits_actor.dropna(subset=['actor_name'])

In [105]:
credits_actor = credits_actor.drop_duplicates(subset='id_movie')

In [106]:
print(credits_actor)

         id_movie        actor_name
0             862         Tom Hanks
1378         8844    Robin Williams
1794        15602    Walter Matthau
1822        31357   Whitney Houston
1922        11862      Steve Martin
...           ...               ...
9479246     30840    Patrick Bergin
9479291    439050      Leila Hatami
9479318    111109      Angel Aquino
9479384     67758     Erika Eleniak
9479459    227506  Iwan Mosschuchin

[42668 rows x 2 columns]


In [107]:
# Se cambia el tipo de dato de la columna 'id_movie' a 'int'
credits_actor['id_movie'] = credits_actor['id_movie'].astype(int)

In [108]:
credits_actor.to_parquet(r'Movies\df_credits_actor.parquet', engine='auto', compression='snappy')